In [1]:
import sys
import h5py
import copy
import pandas as pd
import numpy as np
from utils.get_groups import GetGroups
from utils.paths import SetupPaths
from utils.vectorCorrection import vectorCorrection as vector
from utils.abundance_matching import AbundanceMatching

paths = SetupPaths()

imported h5py...
modules seem to have loaded?


In [61]:
pair_data.close()

In [62]:
# start with ill 135  
    
snapnum = 56
sim = "Illustris"
pair_path = f"{sim}_{snapnum}_10.hdf5"
pair_data = h5py.File(f"{paths.path_pairs}{pair_path}", "r")
# realarray = np.unique(np.array(pair_data['pairs']['dark']['Realization']))

In [64]:
pair_data['Header'].attrs['Redshift']

3.7087742646422353

In [60]:
for key in pair_data['pairs']['dark'].keys():
    print(len(pair_data['pairs']['dark'][key]))
for key in pair_data['pairs']['hydro'].keys():
    print(len(pair_data['pairs']['hydro'][key]))

70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
70308
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [52]:
len(pair_data['pairs']['dark']['Group ID']) == 0

True

pair_data: "Header", "pairs", "unpaired"

pair_data['pairs']: "dark", "hydro"
pair_data['unpaired']: "dark", "hydro"

**keys for unpaired:** *'Group ID', 'Group Mass', 'Group Nsubs', 'Group Radius', 'Sub1 BHMass', 'Sub1 BHMdot', 'Sub1 GasMetallicity', 'Sub1 ID', 'Sub1 Mass', 'Sub1 MassType', 'Sub1 Pos', 'Sub1 SFR', 'Sub1 SFRinRad', 'Sub1 Stellar Mass', 'Sub1 Vel'*

**keys for paired:** *'Group ID', 'Group Mass', 'Group Nsubs', 'Group Radius', 'Realization', 'RelVel', 'Separation', 'Stellar Mass Ratio', 'Sub1 BHMass', 'Sub1 BHMdot', 'Sub1 GasMetallicity', 'Sub1 ID', 'Sub1 Mass', 'Sub1 MassType', 'Sub1 Pos', 'Sub1 SFR', 'Sub1 SFRinRad', 'Sub1 Stellar Mass', 'Sub1 Vel', 'Sub2 BHMass', 'Sub2 BHMdot', 'Sub2 GasMetallicity', 'Sub2 ID', 'Sub2 Mass', 'Sub2 MassType', 'Sub2 Pos', 'Sub2 SFR', 'Sub2 SFRinRad', 'Sub2 Stellar Mass', 'Sub2 Vel', 'TripleFlag'*

In [27]:
num_prims_per_real = []
for realization in realarray:
    # find num of unpaired prims:
    unpaired_stells = np.array(pair_data['unpaired']['dark']['Sub1 Stellar Mass'])
    unpaired_realization = np.array(pair_data['unpaired']['dark']['Realization'])
    
    mask_prims = (unpaired_stells > 0.01) & (unpaired_stells < 0.5)
    mask_real = unpaired_realization == realization

    num_unpairedprims = np.count_nonzero(mask_real & mask_prims)
    
    
    # find num of paired prims:
    paired_stells = np.array(pair_data['pairs']['dark']['Sub1 Stellar Mass'])
    paired_realization = np.array(pair_data['pairs']['dark']['Realization'])
    
    mask_prims = (paired_stells > 0.01) & (paired_stells < 0.5)
    mask_real = paired_realization == realization
    
    num_pairedprims = np.count_nonzero(mask_real & mask_prims)

    tot_numprims = num_unpairedprims + num_pairedprims
    
    print(f"total number of dwarf prims in real {realization} is: {tot_numprims}")
    num_prims_per_real.append(tot_numprims)
    
print(num_prims_per_real)

total number of dwarf prims in real -1 is: 11573
total number of dwarf prims in real 0 is: 11440
total number of dwarf prims in real 1 is: 11449
total number of dwarf prims in real 2 is: 11401
total number of dwarf prims in real 3 is: 11400
total number of dwarf prims in real 4 is: 11509
total number of dwarf prims in real 5 is: 11413
total number of dwarf prims in real 6 is: 11450
total number of dwarf prims in real 7 is: 11423
total number of dwarf prims in real 8 is: 11415
total number of dwarf prims in real 9 is: 11497
total number of dwarf prims in real 10 is: 11409
[11573, 11440, 11449, 11401, 11400, 11509, 11413, 11450, 11423, 11415, 11497, 11409]


In [42]:
quart = np.percentile(num_prims_per_real,[16,84])
print(np.median(num_prims_per_real), quart)

11431.5 [11407.08 11499.88]


In [63]:
pair_data.keys()

<KeysViewHDF5 ['Header', 'pairs']>

In [6]:
pair_data['pairs']['dark'].keys()

<KeysViewHDF5 ['Group ID', 'Group Mass', 'Group Nsubs', 'Group Radius', 'Realization', 'RelVel', 'Separation', 'Stellar Mass Ratio', 'Sub1 BHMass', 'Sub1 BHMdot', 'Sub1 GasMetallicity', 'Sub1 ID', 'Sub1 Mass', 'Sub1 MassType', 'Sub1 Pos', 'Sub1 SFR', 'Sub1 SFRinRad', 'Sub1 Stellar Mass', 'Sub1 Vel', 'Sub2 BHMass', 'Sub2 BHMdot', 'Sub2 GasMetallicity', 'Sub2 ID', 'Sub2 Mass', 'Sub2 MassType', 'Sub2 Pos', 'Sub2 SFR', 'Sub2 SFRinRad', 'Sub2 Stellar Mass', 'Sub2 Vel', 'TripleFlag']>

In [7]:
np.unique(np.array(pair_data['pairs']['dark']['Realization']))

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [14]:
# number of pairs (number of primaries w a companion)
mask = np.array(pair_data['pairs']['dark']['Realization']) == 0 # realization 0
primstar_real = pair_data['pairs']['dark']['Sub1 Stellar Mass'][mask] # realization 0 primaries
mask_prim_dwarfs = (primstar_real > 0.01) & (primstar_real < 0.5) # realization 0 dwarf primaries
print("Number of pairs:",len(np.array(pair_data['pairs']['dark']['Group ID'])))
print("Number of realization 0 pairs:",sum(mask))
print("Number of realization 0 dwarf primaries:",sum(mask_prim_dwarfs))

Number of pairs: 183168
Number of realization 0 pairs: 15264
Number of realization 0 dwarf primaries: 8427


In [15]:
# number of unpaired primary dwarfs
mask = np.array(pair_data['unpaired']['dark']['Realization']) == 0
primstar_real = pair_data['unpaired']['dark']['Sub1 Stellar Mass'][mask]
mask_prim_dwarfs = (primstar_real > 0.01) & (primstar_real < 0.5) 
print("Number of :",len(np.array(pair_data['unpaired']['dark']['Group ID'])))
print("Number of :",sum(mask))
print("Number of :",sum(mask_prim_dwarfs))

Number of : 39792
Number of : 3316
Number of : 3013


In [61]:
import time

start_time = time.time()
print(sum(mask))
print("--- sum() %s seconds ---" % (time.time() - start_time))

start_time = time.time()
print(np.count_nonzero(mask))
print("--- count_nonzero(): %s seconds ---" % (time.time() - start_time))

start_time = time.time()
print(np.bincount(mask))
print("--- bincount(): %s seconds ---" % (time.time() - start_time))


15264
--- sum() 0.017501115798950195 seconds ---
15264
--- count_nonzero(): 0.0004572868347167969 seconds ---
[167904  15264]
--- bincount(): 0.0009844303131103516 seconds ---


In [ ]:
count_nonzero(sum(mask_prim_dwarfs))

In [ ]:
num_prims_per_real = []
for realization in realarray:
    # find num of unpaired prims:
    unpaired_stells = pair_data['unpaired']['dark']['Sub1 Stellar Mass']
    unpaired_realization = pair_data['unpaired']['dark']['Realization']
    
    mask_prims = (unpaired_stells > 0.01) & (unpaired_stells < 0.5)
    mask_real = unpaired_realization == realization

    num_unpairedprims = np.count_nonzero(mask_real & mask_prims)
    
    
    # find num of paired prims:
    paired_stells = pair_data['pairs']['dark']['Sub1 Stellar Mass']
    paired_realization = pair_data['pairs']['dark']['Realization']
    
    mask_prims = (paired_stells > 0.01) & (paired_stells < 0.5)
    mask_real = paired_realization == realization
    
    num_pairedprims = np.count_nonzero(mask_real & mask_prims)

    tot_numprims = num_unpairedprims + num_pairedprims
    
    print(f"total number of dwarf prims in real {realization} is: {tot_numprims}")
    num_prims_per_real.append(tot_numprims)
    
print(num_prims_per_real)

In [ ]:
sum(mask_prim_dwarfs) + sum(unpaired dwarfs in realization 0)

In [51]:
pair_data.close()

In [45]:
# start with ill 135  
    
snapnum = 135
sim = "Illustris"
pair_path = f"{sim}_{snapnum}_10.hdf5"
pair_data = h5py.File(f"{paths.path_pairs}{pair_path}", "a")

In [46]:
pair_data.__delitem__('/unpaired/')

In [47]:
pair_data['pairs'].keys()

<KeysViewHDF5 ['Header', 'pairs']>

In [17]:
try:
    data = pair_data[self.phys]
    red_coor = pair_data['Header'].attrs['Redshift']

except KeyError:
    print(f"snapshot {snapshot}: data does not exist - try another snapshot")

else:
    med_mask = np.array(data["Realization"]) == -1

    
    primary_stell = np.array(data["Sub1 Stellar Mass"])

    # primary stellar mass will be different for dwarf and massive pairs!~
    if self.size == "dwarf":
        primary_analog = (primary_stell > 0.01) & (primary_stell < 0.5)
    elif self.size == "massive":
        primary_analog = (primary_stell > 0.5) & (primary_stell < 10) # this doesn't matter, can be changed! 

    # get major pairs only  
    majors = np.array(data["Stellar Mass Ratio"]) > 1/4
    minors = (np.array(data["Stellar Mass Ratio"]) < 1/4) & (np.array(data["Stellar Mass Ratio"]) > 1/10)
    pair = majors+minors

    minsep = np.array(data["Separation"]) >10
    lowsep = np.array(data["Separation"]) < 50
    highsep = np.array(data["Separation"]) > 50

    lowvel = np.array(data["RelVel"]) < 100
    highvel = np.array(data["RelVel"]) > 100

    countarray = []

    for i in np.arange(-1,10):
        realmask = np.array(data["Realization"]) == i
        counts = np.sum(lowsep & primary_analog & pair & minsep & realmask)
        numprims = np.sum(primary_analog & realmask)
        countarray.append(counts/numprims)

    masks = {"primaries":primary_analog,
             "allpairs":primary_analog & minsep,
             "median": primary_analog & med_mask,
             "major":majors & primary_analog & minsep, 
             "minor":minors & primary_analog & minsep,
             "majormed":majors & med_mask & primary_analog & minsep,
             "minormed":minors & med_mask & primary_analog & minsep,
             "lowsep":lowsep & primary_analog & pair & minsep,
             "highsep":highsep & primary_analog & pair & minsep,
             "lowsepmajor":lowsep & primary_analog & pair & minsep & majors,
             "highsepmajor":highsep & primary_analog & pair & minsep & majors,
             "lowvel":lowvel & primary_analog & pair & minsep,
             "highvel":highvel & primary_analog & pair & minsep,
             "lowvelmajor":lowvel & primary_analog & pair & minsep & majors,
             "highvelmajor":highvel & primary_analog & pair & minsep & majors}

    use_mask = masks[self.subset]
    num_pairs = np.sum(use_mask)
    num_primaries = np.sum(primary_analog)
    ratio_pairs = num_pairs/num_primaries

    if all(use_mask == False):
        print(f"{snapshot} {self.phys} {self.size} no pairs in subset {self.subset}")

    # dictionary  of subhalos in group
    sublist = {"Snapshot":snapnum,
               "Redshift":red_coor,
               "Simulation":self.sim,
               "Number Pairs":np.sum(use_mask),
               "Number Primaries":np.sum(primary_analog),
               "Number Tertiaries":np.sum(primary_analog),
               "Ratio Pairs": num_pairs/num_primaries}

    for key, val in data.items():
        sublist[key] = np.array(val)[use_mask]

NameError: name 'self' is not defined

In [65]:
paths.path_pairs

'/xdisk/gbesla/katiechambe/pears/data/pairs/'

In [ ]:
h5py.close(f"/xdisk/gbesla/katiechambe/pears/data/pairs/Illustris_97_10.hdf5")
for i in np.arange(0,136):
    sim = "Illustris"
    pair_path = f"{sim}_{i}_10.hdf5"
    try:
        pair_data = h5py.File(f"{paths.path_pairs}{pair_path}", "r+")
        dat = pair_data['unpaired']
        dat[0]
    except OSError:
        print("snap 97 open for some reason")
    except:
        print(f'dne in snap{i}')
    pair_data.close()

In [8]:
# pairs = {}
# for it_key1 in pair_data.keys()[1:]: #skip header
#     pairs[it_key1] = {}
#     for it_key2 in pair_data[itkey1].keys():
#         pairs[it_key1][it_key2]